In [2]:
!hdfs dfs -cat /user/clayton/data/juros_selic/juros_selic

"data";"valor"
"01/06/1986";"1,27"
"01/07/1986";"1,95"
"01/08/1986";"2,57"
"01/09/1986";"2,94"
"01/10/1986";"1,96"
"01/11/1986";"2,37"
"01/12/1986";"5,47"
"01/01/1987";"11,00"
"01/02/1987";"19,61"
"01/03/1987";"11,95"
"01/04/1987";"15,30"
"01/05/1987";"24,63"
"01/06/1987";"18,02"
"01/07/1987";"8,91"
"01/08/1987";"8,09"
"01/09/1987";"7,99"
"01/10/1987";"9,45"
"01/11/1987";"12,92"
"01/12/1987";"14,38"
"01/01/1988";"16,78"
"01/02/1988";"18,35"
"01/03/1988";"16,59"
"01/04/1988";"20,25"
"01/05/1988";"18,65"
"01/06/1988";"20,17"
"01/07/1988";"24,69"
"01/08/1988";"22,63"
"01/09/1988";"26,25"
"01/10/1988";"29,79"
"01/11/1988";"28,41"
"01/12/1988";"30,24"
"01/01/1989";"22,97"
"01/02/1989";"18,95"
"01/03/1989";"20,41"
"01/04/1989";"11,52"
"01/05/1989";"11,43"
"01/06/1989";"27,29"
"01/07/1989";"33,15"
"01/08/1989";"35,49"
"01/09/1989";"38,58"
"01/10/1989";"47,70"
"01/11/1989";"48,41"
"01/12/1989";"64,21"
"01/01/1990";"67,60"
"01/02/1990";"82,04"
"01/03/1990";"36,76"
"01/04/1990";"4,23"
"01/05/199

1. Criar um dataframe para ler o arquivo no HDFS /user/<nome/data/juros_selic/juros_selic

In [5]:
from pyspark.sql.functions import *

In [9]:
juros = spark.read.csv("/user/clayton/data/juros_selic/juros_selic", sep=";", header="true")

In [10]:
juros.show(2)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
+----------+-----+
only showing top 2 rows



2. Alterar o formato do campo data para “MM/dd/yyyy”

In [15]:
juros_americano = juros.withColumn("data",from_unixtime(unix_timestamp(col("data"),"dd/MM/yyyy"),"MM/dd/yyyy"))
juros_americano.show(2)

+----------+-----+
|      data|valor|
+----------+-----+
|06/01/1986| 1,27|
|07/01/1986| 1,95|
+----------+-----+
only showing top 2 rows



3. Com uso da função from_unixtime crie o campo “ano_unix”, com a informação do ano do campo data

In [18]:
juros_unixtime = juros_americano.withColumn("ano_unix",from_unixtime(unix_timestamp(col("data"),"dd/MM/yyyy"),"yyyy"))
juros_unixtime.show(3)

+----------+-----+--------+
|      data|valor|ano_unix|
+----------+-----+--------+
|06/01/1986| 1,27|    1986|
|07/01/1986| 1,95|    1986|
|08/01/1986| 2,57|    1986|
+----------+-----+--------+
only showing top 3 rows



4. Com uso de substring crie o campo “ano_str”, com a informação do ano do campo data


In [20]:
juros_substring = juros_unixtime.withColumn("ano_str", substring(col("data"),7,4))
juros_substring.show(2)

+----------+-----+--------+-------+
|      data|valor|ano_unix|ano_str|
+----------+-----+--------+-------+
|06/01/1986| 1,27|    1986|   1986|
|07/01/1986| 1,95|    1986|   1986|
+----------+-----+--------+-------+
only showing top 2 rows



5. Com uso da função split crie o campo “ano_str”, com a informação do ano do campo data

In [21]:
juros_split = juros_substring.withColumn("ano_split", split(col("data"), "/").getItem(2))
juros_split.show(2)

+----------+-----+--------+-------+---------+
|      data|valor|ano_unix|ano_str|ano_split|
+----------+-----+--------+-------+---------+
|06/01/1986| 1,27|    1986|   1986|     1986|
|07/01/1986| 1,95|    1986|   1986|     1986|
+----------+-----+--------+-------+---------+
only showing top 2 rows



6. Salvar no hdfs /user/rodrigo/juros_selic_americano no formato CSV, incluindo o cabeçalho

In [23]:
juros_split.write.csv("/user/clayton/juros_selic", header="true")

In [24]:
!hdfs dfs -ls /user/clayton/juros_selic

Found 2 items
-rw-r--r--   2 root supergroup          0 2022-04-06 20:39 /user/clayton/juros_selic/_SUCCESS
-rw-r--r--   2 root supergroup      13086 2022-04-06 20:39 /user/clayton/juros_selic/part-00000-68a67776-fa05-4150-a0b3-8cd3cdf04894-c000.csv
